In [121]:
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch([{'host':'uct2-es-door.mwt2.org', 'port':9200}])

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [234]:
ind="stashcp-2016.6"

myquery1={
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "query": "*",
          "analyze_wildcard": True
        }
      },
      "filter": {
        "bool": {
          "must": [
            {
              "query": {
                "query_string": {
                  "analyze_wildcard": True,
                  "query": "*a2gb*"
                }
              }
            },
            {
              "range": {
                "timestamp": {
                  "gte": 1466622721000,
                  "lte": 1466674003000,
                  "format": "epoch_millis"
                }
              }
            }
          ],
        }
      }
    }
  },
}

myquery2={
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "query": "*",
          "analyze_wildcard": True
        }
      },
      "filter": {
        "bool": {
          "must": [
            {
              "query": {
                "query_string": {
                  "analyze_wildcard": True,
                  "query": "*a2gb*"
                }
              }
            },
            {
              "range": {
                "timestamp": {
                  "gte": 1466622721000,
                  "lte": 1466674003000,
                  "format": "epoch_millis"
                }
              }
            }
          ],
        }
      }
    }
  },
}

In [309]:
page= es.search(index=ind, body=myquery1, scroll='2m', search_type='scan', size=1)

sid = page['_scroll_id']

scroll_size = page['hits']['total']
results=[]

while (scroll_size > 0):
    #print ("Scrolling...")
    page = es.scroll(scroll_id = sid, scroll = '2m')
    results.append(page['hits']['hits'])
    # Update the scroll ID
    sid = page['_scroll_id']
    # Get the number of results that we returned in the last scroll
    scroll_size = len(page['hits']['hits'])
    #print ("scroll size: " + str(scroll_size))

scroll_size = page['hits']['total']
Res=[]
for i in range(0,scroll_size):
    Res.append(results[i][0]['_source'])
    
print(scroll_size)

df = pd.DataFrame(Res)

del df['destination_space']
del df['xrdcp_version']
del df['tries']
del df['download_size']
del df['filesize']
del df['IP']
del df['xrdcp_exit']

In [411]:
df_UCSDT2=pd.DataFrame(columns=['download_time','filename','host','sitename','status','timestamp'])
for i in range(scroll_size):
    if df.iloc[i,3]=='UCSDT2-C':
        df_UCSDT2=df_UCSDT2.append(df.iloc[[i]])
        
df_UCSDT2=df_UCSDT2.sort_values(by=['timestamp'])
df_UCSDT2['try']=0
f=0
s=0
i=0
length=df_UCSDT2.shape[0]
#print(df_UCSDT2.head(10))    
while i<200:
    df_UCSDT2.iloc[i,6]=1
    df_UCSDT2.iloc[i+1,6]=2
    f+=int(df_UCSDT2.iloc[i,0])
    s+=int(df_UCSDT2.iloc[i+1,0])
    i=i+2
    
#second / first try = speed-up
UCSDT2_speedup=(s/f)
print(UCSDT2_speedup)

0.078832089746585


In [412]:
def speed_up(sitename):
    df_site=pd.DataFrame(columns=['download_time','filename','host','sitename','status','timestamp'])
    for i in range(scroll_size):
        if df.iloc[i,3]==sitename:
            df_site=df_site.append(df.iloc[[i]])
            
    df_site=df_site.sort_values(by=['timestamp'])
    df_site['try']=0
    f=0
    s=0
    i=0
    length=df_site.shape[0]
    #print (length)
    if (length%2==1):
        df_site=df_site[:length-1]
    length=df_site.shape[0]
    host=df_site.iloc[0,2]
    #print (length)
    #print(df_site.head(100))
    while i<length:
        df_site.iloc[i,6]=1
        df_site.iloc[i+1,6]=2
        f+=int(df_site.iloc[i,0])
        s+=int(df_site.iloc[i+1,0])
        i+=2

    #second / first try = speed-up
    speedup=(s/f)
    avg1=(f/length)/1000
    avg2=(s/length)/1000
    print("- speed up: %.2f, avg first try: %.2f sec, avg second try: %.2f sec using host %s" % (speedup, avg1, avg2, host))

#speed_up('UFlorida-HPC')

In [409]:
print("UCSDT2-C:")
speed_up('UCSDT2-C')
print("BNL_ATLAS_1:")
speed_up('BNL_ATLAS_1')
print("UFlorida-HPC:")
speed_up('UFlorida-HPC')
print("UNL (red-gateway2):")
speed_up('red-gateway2')
print("MWT2_CE_UIUC:")
speed_up('MWT2_CE_UIUC')
print("UNC: (Sandhills-CE1):")
speed_up('Sandhills-CE1')
print("SU-OG-CE:")
speed_up('SU-OG-CE')

UCSDT2-C:
- speed up: 0.08, avg first try: 146.30 sec, avg second try: 11.53 sec using host root://xrd-cache-1.t2.ucsd.edu
BNL_ATLAS_1:
- speed up: 0.95, avg first try: 55.16 sec, avg second try: 52.68 sec using host root://data.ci-connect.net
UFlorida-HPC:
- speed up: 0.99, avg first try: 40.15 sec, avg second try: 39.76 sec using host root://data.ci-connect.net
UNL (red-gateway2):
- speed up: 0.24, avg first try: 42.18 sec, avg second try: 10.27 sec using host root://hcc-stash.unl.edu
MWT2_CE_UIUC:
- speed up: 1.30, avg first try: 23.81 sec, avg second try: 30.99 sec using host root://data.ci-connect.net
UNC: (Sandhills-CE1):
- speed up: 0.38, avg first try: 23.83 sec, avg second try: 8.99 sec using host root://hcc-stash.unl.edu
SU-OG-CE:
- speed up: 0.99, avg first try: 102.83 sec, avg second try: 101.83 sec using host root://data.ci-connect.net
